In [1]:
%pip install feast[cassandra]==0.26
%pip install psycopg2-binary

     |████████████████████████████████| 29.0 MB 120 kB/s eta 0:00:01     |████████████████▍               | 14.9 MB 1.4 MB/s eta 0:00:11
     |████████████████████████████████| 12.2 MB 82.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 29.4 MB 149 kB/s  eta 0:00:01
     |████████████████████████████████| 2.8 MB 40.2 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 72 kB 835 kB/s  eta 0:00:01
     |████████████████████████████████| 3.2 MB 56.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 74.0 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 5.3 MB 54.6 MB/s eta 0:00:01
     |████████████████████████████████| 20.5 MB 69.5 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 4.4 MB/s  eta 0:00:01
     |███████████████████████████████

In [2]:
from feast import FeatureStore
store = FeatureStore(repo_path=".")

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage


/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/opt/conda/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [3]:
# !feast apply

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import pandas as pd

In [5]:
loans = pd.read_parquet("loan_table.parquet")

In [6]:
loan_times = loans[["loan_id", "event_timestamp", "created_timestamp"]]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# parquet_name = "loan_approval_baseline.parquet"
# dataset_name = "loan_approval_baseline"

# parquet_name = "loan_approval_2022q3.parquet"
# dataset_name = "loan_approval_2022q3_m1"

parquet_name = "loan_approval_2022q4.parquet"
dataset_name = "loan_approval_2022q4_m1"

In [28]:
q3 = pd.read_parquet(parquet_name)[loans.columns]

In [29]:
q3_without_time = q3[['loan_id', 'dob_ssn', 'zipcode', 'person_age', 'person_income',
       'person_home_ownership', 'person_emp_length', 'loan_intent',
       'loan_amnt', 'loan_int_rate', 'loan_status']]

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
merged = q3_without_time.merge(loan_times, on="loan_id", how="left")

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
loan_features = store.get_feature_service('loan_features')

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source import (
    PostgreSQLSource, SavedDatasetPostgreSQLStorage
)

training_features = store.get_historical_features(
    entity_df=merged, 
    features=loan_features
)

In [33]:
dataset = store.create_saved_dataset(
    from_=training_features,
    name=dataset_name,
    storage=SavedDatasetPostgreSQLStorage(table_ref=dataset_name)
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.8/site-packages/feast/feature_store.py:1182: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [26]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

conn = None
cur = None

try:
    conn = psycopg2.connect("host=cassandra-postgresql.feast-db port=5432 dbname=FEAST_OFFLINE_STORE user=postgres password=postgres")
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS loan_approval_2022q4_m1;')
    print('droped')
except:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


droped


In [ ]:
# 